In [1]:
#import sqlite package
import sqlite3

In [2]:
#connect to the local copy of case study database
conn = sqlite3.connect('sqlite_db_pythonsqlite.db')


In [3]:
#create a cursor
c = conn.cursor()

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! 

In [4]:
query10 = """SELECT facility, SUM(booking_revenue) AS total_revenue
FROM (
	SELECT f.name AS facility, b.slots * guestcost AS booking_revenue
	From Members as m
	INNER JOIN Bookings as b
	ON m.memid = b.memid
	INNER JOIN Facilities as f
	ON b.facid = f.facid
	WHERE b.memid = 0
	UNION
	SELECT f.name AS facility, b.slots * membercost AS booking_revenue
	From Members as m
	INNER JOIN Bookings as b
	ON m.memid = b.memid
	INNER JOIN Facilities as f
	ON b.facid = f.facid
	WHERE b.memid >0) AS combined_revenue
GROUP BY facility
ORDER BY total_revenue DESC;"""


In [5]:
#execute query 10
c.execute(query10)
print(c.fetchmany(5))

[('Massage Room 1', 1659.4), ('Tennis Court 1', 840), ('Squash Court', 637.0), ('Tennis Court 2', 540), ('Massage Room 2', 499.8)]


Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [6]:
query11 = """SELECT DISTINCT CONCAT_WS(' ', m1.firstname, m1.surname) AS Member_name,  
	CASE WHEN m1.recommendedby = "" THEN "Not recommended by anyone"
	ELSE CONCAT_WS(' ', m2.firstname, m2.surname) END AS recommended_by
FROM Members AS m1
LEFT JOIN Members AS m2
ON m1.recommendedby = m2.memid
WHERE m1.memID > 0
ORDER BY m1.surname ASC;"""

In [7]:
#execute query 11
c.execute(query11)
print(c.fetchmany(5))

[('Florence Bader', 'Ponder Stibbons'), ('Anne Baker', 'Ponder Stibbons'), ('Timothy Baker', 'Jemima Farrell'), ('Tim Boothe', 'Tim Rownam'), ('Gerald Butters', 'Darren Smith')]


Q12: Find the facilities with their usage by member, but not guests

In [8]:
query12 = """SELECT f.name AS facility, CONCAT_WS(' ', m.firstname, m.surname) AS Member_name, SUM(b.slots) AS slots_usage, SUM(b.slots*30) AS usage_in_minutes
FROM Bookings AS b
INNER JOIN Members AS m
ON b.memid = m.memid
INNER JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name, m.surname;"""

In [9]:
#execute query 12
c.execute(query12)
print(c.fetchmany(5))

[('Badminton Court', 'Florence Bader', 27, 810), ('Badminton Court', 'Timothy Baker', 51, 1530), ('Badminton Court', 'Tim Boothe', 36, 1080), ('Badminton Court', 'Gerald Butters', 63, 1890), ('Badminton Court', 'Erica Crumpet', 6, 180)]


Q13: Find the facilities usage by month, but not guests

In [10]:
query13 ="""SELECT 
	f.name AS facility, 
    strftime('%m', b.starttime) AS month,
	SUM(b.slots) AS slots_usage, 
	SUM(b.slots*30) AS usage_in_minutes
FROM Bookings AS b
INNER JOIN Facilities AS f
ON b.facid = f.facid
WHERE b.memid > 0
GROUP BY f.name, month;"""

In [11]:
#execute query 14
c.execute(query13)
print(c.fetchmany(5))

[('Badminton Court', '07', 165, 4950), ('Badminton Court', '08', 414, 12420), ('Badminton Court', '09', 507, 15210), ('Massage Room 1', '07', 166, 4980), ('Massage Room 1', '08', 316, 9480)]
